
# **Bank Loan Data**

#### Lucía Poyan, Nuria Olmedilla y Claudia Gemeno

Loan data analysis is a fundamental task for banks and other financial institutions. By using historical data from loan applications, patterns can be identified to predict the likelihood of loan default. This type of analysis enables financial institutions to make informed decisions about whom to grant a loan, based on the applicant's repayment capacity and financial behavior history.

In this practice, we will work with a dataset containing information about clients who applied for loans at a bank. The main task is to apply Exploratory Data Analysis (EDA) techniques to understand the patterns and characteristics of clients that may be related to loan default. Through this process, we aim to identify early warning signs that indicate whether an applicant might fail to repay the loan.

## Problem Definition:

 The objective is to identify the factors contributing to the probability of a client defaulting on a loan.
 By performing exploratory analysis, we will explore key variables in the dataset to find possible correlations
 between client characteristics and default risk. The insights gained from this analysis can be used to improve
 the loan approval process, minimizing the risk of loss for the bank.

 This analysis will address the question: What type of clients are more likely to default on a loan?
 This information will be crucial for developing more accurate strategies in loan allocation and financial risk management.

 The steps to be performed are:

 1. Initial data analysis and preprocessing
 2. Correlations, handling missing values, and outliers
 3. Treatment of categorical variables: encoding
 4. Application of algorithms
 5. Evaluation with the test sample

### **Importing Libraries**

In [1]:
# Required libraries
import os
import pandas as pd
import plotly.express as px

In [2]:
# Display settings for Dataframes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Relative path to the file
pd_credit = pd.read_csv(r'../content/application_data.csv')

### **Future Variables**

Future variables contain information that would not be available at the time of decision-making,
such as when evaluating a loan application. Including such variables would bias the model,
as it would have access to information not available in a real-world scenario.

To identify future variables, we relied on a general evaluation of the context and column names that seem to indicate data collected or calculated after an initial event, in this case, a loan or credit application. This is not 100% certain as it depends on the specific context of the problem and the exact meaning of each variable in our dataset.

After analyzing all variables, none were identified as "future variables" in this dataset.
All variables reflect prior or present client characteristics and behavior.

`SK_ID_CURR` is a unique identifier for an existing loan. It does not contain future information, only present or past records.

`TARGET` is the target variable.

`NAME_CONTRACT_TYPE`: Identifies the type of loan contract, which is a fact established at the beginning of the contract, not future information.

Profile data such as `DAYS_BIRTH`, `CODE_GENDER`, `CNT_CHILDREN`, `AMT_INCOME_TOTAL`, `FLAG_OWN_CAR`, `FLAG_OWN_REALTY`, `NAME_INCOME_TYPE`, `NAME_EDUCATION_TYPE`, `NAME_FAMILY_STATUS`, `NAME_HOUSING_TYPE`, etc., are fixed attributes or change very infrequently and do not depend on future events. They reflect the client's situation at the time of evaluation and are therefore safe to include in the model.


The AMT values are calculated at the time of the loan application:
These amounts are determined based on the requested amount `AMT_CREDIT`, the contract conditions `AMT_ANNUITY`, or the price of the financed good `AMT_GOODS_PRICE`. These values are defined and fixed from the moment the client is evaluated.

Other data, such as `FLAG_EMP_PHONE`, `FLAG_WORK_PHONE`, `FLAG_CONT_MOBILE`, `FLAG_PHONE`, `FLAG_EMAIL`, `FLAG_MOBIL`, among others, are current data based on the information provided by the client.

`REGION_RATING_CLIENT` and `REGION_RATING_CLIENT_W_CITY` are based on current geographical information.

The Credit Bureau queries `AMT_REQ_CREDIT_BUREAU_` show the number of times the institution has checked the client's credit history over different periods (days, weeks, months, quarters, years).

In reality, each query can be considered information obtained before or at the time of the loan application, as part of the credit history. If this information is available at the time of risk evaluation, it is not future data. It only reflects the client's previous behavior in terms of credit access, not payment outcomes.

Finally, we have concluded that all these variables reflect prior or present characteristics and behaviors of the client, not events occurring after the prediction. This means they do not bias the model with information from a period that occurs after the one we aim to analyze.

If there had been any future variables, the procedure to handle them would have been as follows:

```python
# Identified future variables
list_future_variables = [
    'FUTURE_VARIABLE_NAME_1',
    'FUTURE_VARIABLE_NAME_2',
    'FUTURE_VARIABLE_NAME_3',
    'FUTURE_VARIABLE_NAME_4',
    'FUTURE_VARIABLE_NAME_X'
]

# Remove future variables and other irrelevant variables
pd_credit_cleaned = pd_credit.drop(columns=list_future_variables)

# Confirm that the variables have been removed
print("Remaining columns after removing future variables:")
print(pd_credit_cleaned.columns)
```

From this point forward, we would work with the new dataset without future variables: pd_credit_cleaned.

Since there are no future variables, we will continue working with the original dataset "pd_credit."

### **General table analysis**
Dimensions

The first step in understanding the dataset is to examine its dimensions, which includes the number of rows (observations) and columns (variables). This gives an overview of the dataset's size and scope.

In [4]:
print(pd_credit.shape, pd_credit.drop_duplicates().shape)

(307511, 122) (307511, 122)


In [5]:
pd_credit

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

Data types

In [6]:
pd_credit.dtypes.to_dict()

{'SK_ID_CURR': dtype('int64'),
 'TARGET': dtype('int64'),
 'NAME_CONTRACT_TYPE': dtype('O'),
 'CODE_GENDER': dtype('O'),
 'FLAG_OWN_CAR': dtype('O'),
 'FLAG_OWN_REALTY': dtype('O'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE': dtype('O'),
 'NAME_INCOME_TYPE': dtype('O'),
 'NAME_EDUCATION_TYPE': dtype('O'),
 'NAME_FAMILY_STATUS': dtype('O'),
 'NAME_HOUSING_TYPE': dtype('O'),
 'REGION_POPULATION_RELATIVE': dtype('float64'),
 'DAYS_BIRTH': dtype('int64'),
 'DAYS_EMPLOYED': dtype('int64'),
 'DAYS_REGISTRATION': dtype('float64'),
 'DAYS_ID_PUBLISH': dtype('int64'),
 'OWN_CAR_AGE': dtype('float64'),
 'FLAG_MOBIL': dtype('int64'),
 'FLAG_EMP_PHONE': dtype('int64'),
 'FLAG_WORK_PHONE': dtype('int64'),
 'FLAG_CONT_MOBILE': dtype('int64'),
 'FLAG_PHONE': dtype('int64'),
 'FLAG_EMAIL': dtype('int64'),
 'OCCUPATION_TYPE': dtype('O'),
 'CNT_FAM_MEMB

### **Target variable exploration and treatment**

The target variable represents the outcome we aim to predict, such as whether a client defaults on a loan. Understanding its distribution is crucial for evaluating the balance of the dataset and determining if additional preprocessing is necessary.

In [7]:
pd_plot_target = pd_credit['TARGET'].value_counts(normalize=True).mul(100).rename('percent').reset_index()
pd_plot_target.rename(columns={'index': 'TARGET'}, inplace=True)

pd_plot_target_conteo = pd_credit['TARGET'].value_counts().rename('count').reset_index()
pd_plot_target_conteo.rename(columns={'index': 'TARGET'}, inplace=True)

pd_plot_target_pc = pd.merge(pd_plot_target, pd_plot_target_conteo, on='TARGET', how='inner')

print(pd_plot_target_pc)

   TARGET    percent   count
0       0  91.927118  282686
1       1   8.072882   24825


In [8]:
fig = px.bar(pd_plot_target_pc, x='TARGET', y='percent', text='count')

fig.update_layout(
    title='Target variable distribution',
    xaxis_title='TARGET',
    yaxis_title='Percentage (%)',
    template='plotly_white'
)

fig.show()

The X-axis represents the possible values of the TARGET variable, which in this case are 0 (clients WITHOUT payment difficulties) and 1 (clients WITH payment difficulties).

This means:
- If `TARGET` = 0: Clients fulfilled their payments.
- If `TARGET` = 1: Clients experienced defaults or financial issues.

The Y-axis shows the percentage of observations for each class.

Class 0 (without payment difficulties) has a higher proportion, representing approximately 90% of the total.
Class 1 (with payment difficulties) is significantly smaller, around 10% of the total.

The chart highlights an issue of imbalanced classes, common in financial datasets where most clients do not experience payment problems (0), while default cases (1) are a minority.

### **Threshold Selection for Rows and Columns to Remove Missing Values**

In [9]:
# Calculate missing values per column and per row
pd_series_null_columns = pd_credit.isnull().sum().sort_values(ascending=False)
pd_series_null_rows = pd_credit.isnull().sum(axis=1).sort_values(ascending=False)
print(pd_series_null_columns.shape, pd_series_null_rows.shape)

(122,) (307511,)


In [10]:
# Create DataFrames to store information about missing values
pd_null_columns = pd.DataFrame(pd_series_null_columns, columns=['columns_nulls'])
pd_null_rows = pd.DataFrame(pd_series_null_rows, columns=['rows_nulls'])

In [11]:
# Add columns with the percentage of missing values
pd_null_columns['percentage_columns'] = pd_null_columns['columns_nulls'] / pd_credit.shape[0]
pd_null_columns = pd_null_columns.sort_values(by='percentage_columns', ascending=False)
pd_null_rows['percentage_rows'] = pd_null_rows['rows_nulls'] / pd_credit.shape[1]
pd_null_rows = pd_null_rows.sort_values(by='percentage_rows', ascending=False)

In [12]:
# Show initial statistics
print(f"Initial dataset dimensions: {pd_credit.shape}")

print(pd_null_columns.head())
print(pd_null_rows.head())

Initial dataset dimensions: (307511, 122)
                          columns_nulls  percentage_columns
COMMONAREA_MEDI                  214865            0.698723
COMMONAREA_MODE                  214865            0.698723
COMMONAREA_AVG                   214865            0.698723
NONLIVINGAPARTMENTS_MODE         213514            0.694330
NONLIVINGAPARTMENTS_MEDI         213514            0.694330
        rows_nulls  percentage_rows
185713          61              0.5
69707           61              0.5
244833          61              0.5
150206          61              0.5
269786          61              0.5


This code displays the percentage of missing values in each row and column in descending order. Since the highest percentage, both in rows and columns, does not exceed 90%, we do not need to remove anything.

If there had been any percentages greater than 90%, the following procedure would have been applied:

```python
# Define the threshold for removing columns and rows
threshold_columns = 0.9
threshold_rows = 0.9

# Filter columns with less than 90% missing values
list_vars_not_null = list(pd_null_columnas[pd_null_columnas['column_percentage'] < threshold_columns].index)
pd_credit_filtered_columns = pd_credit.loc[:, list_vars_not_null]
print(f"Dimensions after filtering columns with more than {threshold_columns*100}% missing values: {pd_credit_filtered_columns.shape}")

# Filter rows with less than 90% missing values
pd_credit_filtered = pd_credit_filtered_columns[pd_credit_filtered_columns.isnull().sum(axis=1) / pd_credit_filtered_columns.shape[1] < threshold_filas]
print(f"Dimensions after filtering rows with more than {threshold_filas*100}% missing values: {pd_credit_filtered.shape}")
```

### **Types: Categorical and Numerical Variables**

We will calculate the number of unique values for each variable in the dataset. This process will give us a general idea of which variables might be categorical and which are numerical. After performing the calculations, we have set a threshold of 50 unique values, as the dataset contains a considerable number of rows, and we consider this threshold appropriate for the initial distinction. This threshold may be adjusted later as we progress in the analysis.

Practically speaking, variables with fewer than 50 unique values will be considered categorical, while those with more than 50 unique values will be classified as numerical. This is based on the premise that categorical variables tend to have a limited number of distinct values, whereas numerical variables typically exhibit greater diversity in values.

When reviewing the variables initially classified as categorical, we noticed that some of them, despite having fewer than 50 unique values, are actually numerical variables. For example, the variable `OBS_30_CNT_SOCIAL_CIRCLE` contains only 33 unique values but represents the number of observations in the client's social environment with observable delinquency of 30 days (DPD). This clearly indicates that it is a numerical variable.

Based on this analysis, we have decided to reconsider the classification of this and other similar variables, reassigning them as numerical since their nature and context suggest they should be treated as such, regardless of the number of unique values they have.

In [13]:
dict_nunique = {col: pd_credit[col].nunique() for col in pd_credit.columns}
filtered_dict = {key: value for key, value in dict_nunique.items() if value < 50}

list_var_cat = list(filtered_dict.keys())
list_var_continuous = [col for col in pd_credit.select_dtypes(include='number').columns if col not in list_var_cat]

manual_numeric_vars = [
    'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_MON',
    'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_YEAR',
    'OBS_30_CNT_SOCIAL_CIRCLE',
    'DEF_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE',
    'ELEVATORS_MODE',
    'ENTRANCES_MODE',
    'FLOORSMAX_MODE',
    'FLOORSMIN_MODE',
    'ELEVATORS_MEDI',
    'ENTRANCES_MEDI',
    'FLOORSMAX_MEDI',
    'FLOORSMIN_MEDI',
    'HOUR_APPR_PROCESS_START',
    'CNT_FAM_MEMBERS',
    'CNT_CHILDREN'
]

list_var_cat = [col for col in list_var_cat if col not in manual_numeric_vars]
list_var_continuous += manual_numeric_vars


print("Categorical variables:", list_var_cat)
print("Numerical variables:", list_var_continuous)

Categorical variables: ['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',

In [14]:
pd_credit[list_var_cat] = pd_credit[list_var_cat].astype("category")
pd_credit[list_var_continuous] = pd_credit[list_var_continuous].astype(float)

print(pd_credit.dtypes)

SK_ID_CURR                       float64
TARGET                          category
NAME_CONTRACT_TYPE              category
CODE_GENDER                     category
FLAG_OWN_CAR                    category
FLAG_OWN_REALTY                 category
CNT_CHILDREN                     float64
AMT_INCOME_TOTAL                 float64
AMT_CREDIT                       float64
AMT_ANNUITY                      float64
AMT_GOODS_PRICE                  float64
NAME_TYPE_SUITE                 category
NAME_INCOME_TYPE                category
NAME_EDUCATION_TYPE             category
NAME_FAMILY_STATUS              category
NAME_HOUSING_TYPE               category
REGION_POPULATION_RELATIVE       float64
DAYS_BIRTH                       float64
DAYS_EMPLOYED                    float64
DAYS_REGISTRATION                float64
DAYS_ID_PUBLISH                  float64
OWN_CAR_AGE                      float64
FLAG_MOBIL                      category
FLAG_EMP_PHONE                  category
FLAG_WORK_PHONE 

### **Initial preprocessing of some variables**

At this point, we will modify some aspects of the DataFrame to make it cleaner and more readable. First, we will convert all column names to lowercase to maintain a uniform convention. Additionally, we will remove any leading and trailing whitespace from all string values in columns of type "object."

Furthermore, we will transform the variable `WEEKDAY_APPR_PROCESS_START`. Instead of having the days of the week written as text, we will replace them with numbers representing their order (Monday = 1, Tuesday = 2, etc.). This will simplify later encoding, allowing us to represent the days as "Weekday_1," "Weekday_2," etc., which can be useful for subsequent analysis.

In general, we have not identified other aspects requiring cleaning at this moment, so we consider this DataFrame preprocessed and ready for handling missing values, outlier detection, correlation calculations, and other analyses.

In [15]:
pd_credit.columns = pd_credit.columns.str.lower()

pd_credit = pd_credit.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

weekday_mapping = {
    'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3,
    'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7
}

pd_credit['weekday_appr_process_start'] = pd_credit['weekday_appr_process_start'].map(weekday_mapping)


In [16]:
pd_credit.head()

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,years_build_mode,commonarea_mode,elevators_mode,entrances_mode,floorsmax_mode,floorsmin_mode,landarea_mode,livingapartments_mode,livingarea_mode,nonlivingapartments_mode,nonlivingarea_mode,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002.0,1,Cash loans,M,N,Y,0.0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461.0,-637.0,-3648.0,-2120.0,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,3,10.0,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003.0,0,Cash loans,F,N,N,0.0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765.0,-1188.0,-1186.0,-291.0,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,1,11.0,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004.0,0,Revolving loans,M,Y,Y,0.0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046.0,-225.0,-4260.0,-2531.0,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,1,9.0,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [17]:
pd_credit.shape

(307511, 122)

In [18]:
# Relative path
pd_credit.to_csv(r'../content/pd_data_initial_preprocessed.csv', index=False)


In this notebook, we conducted a thorough analysis of a bank loan application dataset with the goal of identifying patterns related to the probability of loan payment default by customers. Below is a summary of the most relevant stages:

1. General Dataset Analysis: We explored the dimensions and data types of the dataset, noting a clear class imbalance (90% of customers without payment difficulties and 10% with difficulties).

2. Detection of Future Variables: We analyzed the available variables and confirmed that none of them represented future data, ensuring the validity of the dataset for modeling.

3. Analysis of Missing Values: We checked that the missing values in the columns did not exceed the critical threshold, allowing us to maintain the integrity of the dataset without losing valuable information.

4. Classification of Categorical and Numeric Variables: We adjusted the initial classifications of some variables to ensure proper treatment according to their nature.

5. Initial Preprocessing: We standardized column names, removed whitespace, and transformed some variables into appropriate numeric formats.

This analysis allowed us to identify the main challenges of the dataset, such as class imbalance, and prepare the data for future analysis and modeling stages. In conclusion, we established a solid foundation for building predictive models, ensuring the quality and consistency of the available information.